# Analisis de Job Shift
* En base a dataset de Kagle

### Importar Librerias

In [1]:
import pandas as pd

## 0 Entender el caso de negocio
* Analizando los factores que ayudan a hacer el cambio a un puesto de HR analytics
1. Employee_1D: codigo unico del empleado
2. city: ciudad ficticia, no exite mas información de la ciudad como tamaño o nivel de ingresos
3. gender: Femenino, masculino, otro (contiene nulls)
4. HR Analytics: Experiencia en HR analytics (tiene o no tiene)
5. University Grade: grado universitario, Grade 1 es el max
6. education_level: Nivel educativo (contiene nulls)
7. Has HR expereince: Tiene o no tiene experiencia en HR
8. experience: años de experiencia (contiene nulls)
9. Job changes: cambios de trabajo por candidato (contiene nulls)
10. Training hours: Horas de entrenamiento del candiato (80% aprox null)
11. company_size: Nro empleados en la empresa donde trabaja (contiene nulls)
12. Get HR Analytics Job? : Obtendra o no un trabajo de HR analytics

#### Objetivo: va ser encontrar qué es lo que influye en que un candidatos pueda obtener un empleo como HR analytics


## 1 Extraer data y forma

In [5]:
#extraigo la base desde un csv
df = pd.read_csv('HR analytics.csv')

In [6]:
# conociendo la forma del df
# 2129 registros y 12 columnas
df.shape

(2129, 12)

In [7]:
# Revisando las columnas que tiene
df.columns

Index(['Employee_1D', 'city', 'gender', 'HR Analytics', 'University Grade',
       'education_level', 'Has HR expereince', 'experience', 'Job changes',
       'Training hours', 'company_size', 'Get HR Analytics Job? '],
      dtype='object')

In [8]:
# Revisnaod el tipo de dato de cada columna
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Employee_1D             2129 non-null   int64  
 1   city                    2129 non-null   object 
 2   gender                  1621 non-null   object 
 3   HR Analytics            2129 non-null   object 
 4   University Grade        2129 non-null   object 
 5   education_level         2077 non-null   object 
 6   Has HR expereince       2129 non-null   object 
 7   experience              2124 non-null   object 
 8   Job changes             2089 non-null   object 
 9   Training hours          29 non-null     float64
 10  company_size            1507 non-null   object 
 11  Get HR Analytics Job?   2129 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 199.7+ KB


In [19]:
# check unique values by colum
print('valores genero', df['gender'].unique())
print('valores HR Analytics', df['HR Analytics'].unique())
print('valores University Grade', df['University Grade'].unique())
print('valores education_level', df['education_level'].unique())
print('valores experience', df['experience'].unique())
print('valores Job changes', df['Job changes'].unique())
print('valores Training hours', df['Training hours'].unique())
print('valores company_size', df['company_size'].unique())


valores genero ['Male' 'Female' nan 'Other']
valores HR Analytics ['Has relevent experience' 'No relevent experience']
valores University Grade ['Grade 2' 'Grade 3' 'Grade 1']
valores education_level ['Graduate' 'High School' 'Masters' nan 'Phd' 'Primary School']
valores experience ['9' '5' '<1' '11' '>20' '10' '14' '3' '20' '8' '4' '13' '2' '6' '7' '1'
 '19' '15' '16' nan '17' '18' '12']
valores Job changes ['1' 'never' '>4' '2' '4' '3' nan]
valores Training hours [ 98.   8.  86.  69.  97.  51. 114.  67. 105.   1.   0. 109.  79. 148.
   5. 121.  39.  23. 120.  90.  82. 135.  59. 150.  70.  21.  10.  99.
  89.  nan]
valores company_size ['<10' nan '1-10' '10000+' '100-500' '50-99' '1000-4999' '500-999'
 '5000-9999']


## 2 Analizando Datos faltantes

* En que columnas tengo faltantes
    * gender (Podria reemplazarlo con la moda, pero es necesario ver si esto afecta a la pregunta final)
    * Education Level (Podria enlazarlo con  univerity grade) 
    * Expercience (solo 5 nan no afectaria mucho)
    * Job changes (podria enlazarlo con la experiencia)
    * Trainning hous (demasiados nan, puede que no ayude esta variable) 98%
    * Company size (28% sde datos faltantes)

In [20]:
df.isnull().sum()

Employee_1D                  0
city                         0
gender                     508
HR Analytics                 0
University Grade             0
education_level             52
Has HR expereince            0
experience                   5
Job changes                 40
Training hours            2100
company_size               622
Get HR Analytics Job?        0
dtype: int64

### Entendiendo Gender para analizar los faltantes

* El gender esta desbalanceado por tener casi el 70% de valores mal

In [33]:
df.groupby(by=['gender'], as_index=False).agg(n = ('Employee_1D', 'count')).sort_values(by='n', ascending = False).n / 2129

1    0.685768
0    0.064349
2    0.011273
Name: n, dtype: float64